# Jupyter Notebook for the evaluation of nucleon PDFs from Distillation correlation functions

## Initialization

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as scipyOpt
import math, itertools as it
import matplotlib as mpl ; import matplotlib.pyplot as plt ; plt.ion()
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetiva']})
rc('text', usetex=True)

print ("Done")

## Directories, Parameters, Constants

### Generic definitions

In [ ]:
def dispTag(d):
    dL = lambda i: ("z" if i != 0 else "") + ("+" if i > 0 else "") + str(i)
    return 'disp_%s'%(dL(d))

def MomTag(zMom):
    mSgn = lambda i: ("+" if i > 0 else "") + str(i)
    return 'pz.%s'%(mSgn(zMom))

def phMomTpl(phTag,zMom):
    return (phTag,MomTag(zMom))

def momFileTag(mVec):
    mFTag = 'momXYZ.%d.%d.%d' % (mVec[0],mVec[1],mVec[2])
    return mFTag
#-----------------------------------

phaseTypes    = ['unphased']
SrcSnkOpTypes = ['single']
maxDisp = 8
dispLen = np.arange(-maxDisp,maxDisp+1)


# Parameters for unphased correlators
phTS = 'unphased'
opTS = 'single'

# Note: Every combination of momentum, t0 and t_sink is considered a 'dataset'
ZmomVal   = {(phTS,opTS): [0,1,2,3,-1,-2,-3]} # Available momenta values
TsinkList = {(phTS,opTS): [4,6,8,10,12,14]}   # Tsink Values
T0List    = {(phTS,opTS): [0,16,32,48]}       # T0 Values

# Insertion gamma matrices
gMatList = ['gt',
            'gxg5','gyg5','gzg5','gtg5',
            'gxgy','gxgz','gxgt','gygz','gygt','gzgt']

# (Maximum) Number of configs
Ncfg_all  = 349

# Base Directory of two- and three-point correlation functions
corr3ptMainDir = {(phTS,opTS): '3pt_data_%s'%(phTS)}
corr2ptMainDir = {(phTS,opTS): '../2pt_corr_effEnergy/2pt_data_%s'%(phTS)}

momTags_Dict = {(phTS,opTS): []}
momVector_Dict  = {}
Ncfg_Dict_3pt = {}
Ncfg_Dict_2pt = {}
existsDataSet_3pt = {}
existsDataSet_2pt = {}
for zmom in ZmomVal[(phTS,opTS)]:
    mTag = MomTag(zmom)
    momTags_Dict[(phTS,opTS)].append(mTag)
    momVector_Dict[mTag] = np.array([0,0,zmom])
    
    for tsnk in TsinkList[(phTS,opTS)]:
        dTpl = (phTS,opTS,mTag,tsnk)
        Ncfg_Dict_3pt[dTpl] = Ncfg_all  # All datasets have the same number of configs
        for t0 in T0List[(phTS,opTS)]:
            dTpl3pt = (phTS,opTS,mTag,tsnk,t0)
            existsDataSet_3pt[dTpl3pt] = True  # Whether all possible datasets exist for the three-point function

    dTpl2 = (phTS,opTS,mTag)
    Ncfg_Dict_2pt[dTpl2] = Ncfg_all            
    for t0 in T0List[(phTS,opTS)]:
        dTpl2pt = (phTS,opTS,mTag,t0)
        existsDataSet_2pt[dTpl2pt] = True  # Whether all possible datasets exist for the two-point function
#----------------------------------------------


# Some data for the two-point function do not exist for now, skip those
for zmom in [3,-1,-2,-3]:
    mTag = MomTag(zmom)
    for t0 in T0List[(phTS,opTS)]:
        dTpl2pt = (phTS,opTS,mTag,t0)
        existsDataSet_2pt[dTpl2pt] = False

zmom = 0
t0 = 48
mTag = MomTag(zmom)
dTpl2pt = (phTS,opTS,mTag,t0)
existsDataSet_2pt[dTpl2pt] = False

print(existsDataSet_2pt)
#----------------------------------------------



# Which datasets to analyze (if there's at least one t0 for each momentum and tsink that's OK)
analyzeDataSet = {}
for mTag in momTags_Dict[(phTS,opTS)]:
    for tsnk in TsinkList[(phTS,opTS)]:
        adTpl = (phTS,opTS,mTag,tsnk)
        for t0 in T0List[(phTS,opTS)]:
            dTpl3pt = (phTS,opTS,mTag,tsnk,t0)
            dTpl2pt = (phTS,opTS,mTag,t0)
            if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                analyzeDataSet[adTpl] = True
            else:
                analyzeDataSet[adTpl] = False
            break
#----------------------------------------------


# Number of time-slices for the two-point function
Nt_Dict_2pt = {phMomTpl(phTS,0): 16,
               phMomTpl(phTS,1): 21,
               phMomTpl(phTS,2): 21}



print('Momentum Tags:\n', momTags_Dict)
print('\n')
print('Momentum Vectors:\n', momVector_Dict)

print('\nDone\n')

### Read/define source-insertion-sink operators

In [ ]:

# Interpolating operators  for 3-point functions         
nRow3pt_Dict = {'single': {}}
nRow2pt_Dict = {'single': {}}
srcOpList3pt_Dict = {'single': {}}
snkOpList3pt_Dict = {'single': {}}
srcOpList2pt_Dict = {'single': {}}
snkOpList2pt_Dict = {'single': {}}
for zmom in ZmomVal[(phTS,opTS)]:
    pmTpl = phMomTpl(phTS,zmom)

    # 2-point function interpolators have one row for zmom = 0
    if zmom == 0:
        nRow2pt_Dict['single'][pmTpl] = 1
    else:
        nRow2pt_Dict['single'][pmTpl] = 2
    
    # 3-point function interpolators have two-rows for all momenta
    nRow3pt_Dict['single'][pmTpl] = 2

    if zmom == 0:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
    else:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
#----------------------------------------------


# Insertion operators list, that's a common file among all momenta
insOpFile_Dict = {phTS: '../pPDF/insert_gamma_row.DA.list'}

# Read insertion operators
insOptrList = {}
insOpFname = insOpFile_Dict[phTS]
insOptrList[phTS] = {'name': [], 'row': []}
with open(insOpFname) as fp:
    line = fp.readlines()
    for t in line:
        insOptrList[phTS]['name'].append(t.split()[0])
        insOptrList[phTS]['row'].append(int(t.split()[1]))

# Create dictionary to map actual gamma matrices to operator names and rows
gMatMap = {phTS: {'gt'  : ('b_b0xDA__J0_A1pP', 1),
                  'gxg5': ('a_a1xDA__J1_T1pM', 1),
                  'gyg5': ('a_a1xDA__J1_T1pM', 3),
                  'gzg5': ('a_a1xDA__J1_T1pM', 2),
                  'gtg5': ('pion_pion_2xDA__J0_A1mM', 1),
                  'gxgy': ('b_b1xDA__J1_T1pP', 2),
                  'gxgz': ('b_b1xDA__J1_T1pP', 3),
                  'gxgt': ('rho_rho_2xDA__J1_T1mP', 3),
                  'gygz': ('b_b1xDA__J1_T1pP', 1),
                  'gygt': ('rho_rho_2xDA__J1_T1mP', 1),
                  'gzgt': ('rho_rho_2xDA__J1_T1mP', 2)}}
#----------------------------------------------

print('Insertion operator List:\n', insOptrList[phTS])

print('\nDone\n')

### Ensemble-related definitions

In [ ]:
# Ensemble parameters
L = 32
T = 64

binsize=1
#------------------------------------------------

# Nucleon, pion mass
alat_fm = 0.094
alat_iGeV = alat_fm / 0.1973
mpi_GeV = 0.350
mN_GeV = 1.123
amN = mN_GeV * alat_iGeV

# Dispersion relation
Pi = np.pi
def dispRel(am,k):
    ap = 2*Pi/L * k
    return np.sqrt(am**2 + np.dot(ap,ap))

print('Done\n')

### Jackknife-related functions

In [ ]:
def jackknife_ave(bins,Nbins,tL):

    Jax = 0
    
    if(tL==1):
        ave = np.mean(bins)

        sqsum = sum(map(lambda x:x*x,ave-bins))
        fac = (Nbins -1) / float(Nbins)
        err = np.sqrt(fac*sqsum)
    else:
        ave = np.mean(bins, axis=Jax)

        err = np.zeros(tL)
        for i in range(tL):
            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i].real))
            fac = (Nbins -1) / float(Nbins)
            err[i] = np.sqrt(fac*sqsum)        
            
    return (ave,err)
#-------------------------------------------------------

def jackknife_binning(corr, binsize=1):
    
    Jax=0
    
    # Define the number of bins
    mod   = Ncfg%binsize
    Nbins = int((Ncfg - mod) / binsize)
    bins = np.zeros((Nbins,Nt))

    csum = np.sum(corr,axis=Jax) # Sum w.r.t to the configurations
    for t in range(Nt):
        for m in np.arange(1,mod+1):
            csum[t] -= corr[Ncfg-m,t]  # Throw away data in case there is modulo
            
        for b in range(Nbins):
            bsum = 0
            for k in range(binsize):
                bsum += corr[b*binsize+k,t]
                
            bins[b][t] = (csum[t] - bsum) / float(Ncfg - binsize - mod) # Bin averages for each bin,t

    return bins, Nbins
#-------------------------------------------------------

print('Done\n')

## Read three-point functions, perform Jackknife sampling

In [ ]:
def get3ptFileName(phT,t0Tag,tsnkTag,src,snk,srow,ins,insRow,mFTag,d):
    filePre = 'corr_3pt.baryon.n64'
    
    srcTag = 'src_%s_%d'%(src,srow)
    snkTag = 'snk_%s_%d'%(snk,srow)
    insTag = 'ins_%s_%d'%(ins,insRow)
    
    dTag = dispTag(d)
    
    FileName = '%s.%s.%s.%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,tsnkTag,srcTag,snkTag,insTag,dTag,mFTag)
    
    return FileName
#--------------------------

def get3ptFileDir(mainDir,t0Tag,mFTag,tsnkTag):
    return '%s/%s/%s/%s' % (mainDir,t0Tag,mFTag,tsnkTag)
#--------------------------------------------------------------


c3pt_Allbins   = {}
c3pt_Allave    = {}
c3pt_rowAvg = {}
c3pt_rawAllave = {}

c3pt_bins   = {}
c3pt_ave    = {}

c3pt_rawAve = {}

Nbins_3pt_Dict = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow3pt_Dict[opT][pmTpl]

            srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
            snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

            for tsnk in TsinkList[(phT,opT)]:
                tsnkTag = 'tsnk_%d'%(tsnk)
                dTpl = (phT,opT,mTag,tsnk)
                Ncfg = Ncfg_Dict_3pt[dTpl]
                Nt = tsnk
                
                # Determine number of t0 we consider
                Nt0 = 0
                for t0 in T0List[(phT,opT)]:
                    dTpl3pt = (phT,opT,mTag,tsnk,t0)
                    dTpl2pt = (phTS,opTS,mTag,t0)
                    if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                        Nt0 += 1

                for isrc, src in enumerate(srcOpList3pt):
                    for isnk, snk in enumerate(snkOpList3pt):
                        for gMat in gMatList:
                            insOpName = gMatMap[phT][gMat][0]
                            insOpRow  = gMatMap[phT][gMat][1]

                            for d in dispLen:
                                ckey = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d) # All info in one tuple
                                
                                c3pt_rawAll = np.zeros((Ncfg,Nt,nRow,Nt0),dtype=np.complex128)

                                for it0,t0 in enumerate(T0List[(phT,opT)]):
                                    dTpl3pt = (phT,opT,mTag,tsnk,t0)
                                    dTpl2pt = (phTS,opTS,mTag,t0)
                                    if existsDataSet_3pt[dTpl3pt] and existsDataSet_2pt[dTpl2pt]:
                                        t0Tag = 't0_%d'%(t0)

                                        for ir,irow in enumerate(range(1,nRow+1)):
                                            ckeyA = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,t0,irow) # All info in one tuple

                                            c3ptFileDir  = get3ptFileDir(corr3ptMainDir[(phT,opT)],t0Tag,mFTag,tsnkTag)
                                            c3ptFileName = get3ptFileName(phT,t0Tag,tsnkTag,src,snk,irow,
                                                                          insOpName,insOpRow,mFTag,d)               
                                            c3ptFile = '%s/%s' % (c3ptFileDir,c3ptFileName)

                                            with open(c3ptFile) as fp:
                                                line = fp.readlines()
                                                c = 0
                                                for n in line:
                                                    it   = c%Nt
                                                    icfg = c//Nt
                                                    c3pt_rawAll[icfg,it,ir,it0] = complex(np.float64(n.split()[1]),
                                                                                            np.float64(n.split()[2]))
                                                    c += 1
                                            # Reading file

                                            # Jackknife average on All three-point functions
                                            c3pt_Allbins[ckeyA], Nbins_3pt_Dict[dTpl] = jackknife_binning(c3pt_rawAll[:,:,ir,it0].real)
                                            Nbins_3pt = Nbins_3pt_Dict[dTpl]
                                            c3pt_Allave[ckeyA] = jackknife_ave(c3pt_Allbins[ckeyA],Nbins_3pt,tL=Nt)
                                        # for-irow

                                        # Standard average and standard error (still average over rows)
                                        # Might need this for constructing covariance matrix
                                        ckeyB = (opT,phT,mTag,tsnk,isrc,isnk,gMat,d,t0)
                                        c3pt_rowAvg[ckeyB] = np.mean(c3pt_rawAll[:,:,:,it0],axis=2,dtype=np.complex128) # Average over rows
                                        c3pt_rawAllave[ckeyB] = (np.mean(c3pt_rowAvg[ckeyB],axis=0),
                                                                    np.std(c3pt_rowAvg[ckeyB],axis=0)/np.sqrt(Ncfg))
                                    else:
                                        print('### Skipping dataset:', dTpl3pt)
                                # for-t0

                                # Average over rows and t0
                                c3pt_Tmp1 = np.mean(c3pt_rawAll,axis=3,dtype=np.complex128) # over t0
                                c3pt_Tmp2 = np.mean(c3pt_Tmp1  ,axis=2,dtype=np.complex128) # over rows

                                # Standard average and standard error
                                c3pt_rawAve[ckey] = (np.mean(c3pt_Tmp2,axis=0),np.std(c3pt_Tmp2,axis=0)/np.sqrt(Ncfg))

                                # Jackknife-average on t0- and row-average            
                                c3pt_bins[ckey], Nbins_3pt = jackknife_binning(c3pt_Tmp2.real)
                                c3pt_ave[ckey] = jackknife_ave(c3pt_bins[ckey],Nbins_3pt,tL=Nt)
                            # for-displacement
                # for-src-snk-ins
                print('Done: Momentum %s , t_sink = %d'%(mTag,tsnk))
            # for tsnk
        # for momentum
        
           
print('\nDone\n')

## Read two-point functions, perform Jackknife sampling

In [ ]:
def get2ptFileName(phT,t0Tag,src,snk,srow,mFTag):
    filePre = 'corr_2pt.baryon.n64'
    
    srcTag = 'src_%s_%d'%(src,srow)
    snkTag = 'snk_%s_%d'%(snk,srow)
    
    FileName = '%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,srcTag,snkTag,mFTag)
    
    return FileName
#--------------------------

def get2ptFileDir(mainDir,t0Tag,mFTag):
    return '%s/%s/%s' % (mainDir,t0Tag,mFTag)
#--------------------------------------------------------------


c2pt_Allbins   = {}
c2pt_Allave    = {}
c2pt_rowAvg = {}
c2pt_rawAllave = {}

c2pt_bins   = {}
c2pt_ave    = {}

c2pt_rawAve = {}

Nbins_2pt_Dict = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow2pt_Dict[opT][pmTpl]

            srcOpList2pt = srcOpList2pt_Dict[opT][pmTpl]        
            snkOpList2pt = srcOpList2pt_Dict[opT][pmTpl]

            dTpl = (phT,opT,mTag)
            Ncfg = Ncfg_Dict_2pt[dTpl]

            Nt = Nt_Dict_2pt[pmTpl]

            # Determine number of t0 we consider
            Nt0 = 0
            for t0 in T0List[(phT,opT)]:
                dTpl2pt = (phTS,opTS,mTag,t0)
                if existsDataSet_2pt[dTpl2pt]:
                    Nt0 += 1

            for isrc, src in enumerate(srcOpList2pt):
                for isnk, snk in enumerate(snkOpList2pt):
                    ckey = (opT,phT,mTag,isrc,isnk) # All info in one tuple

                    c2pt_rawAll = np.zeros((Ncfg,Nt,nRow,Nt0),dtype=np.complex128)

                    for it0,t0 in enumerate(T0List[(phT,opT)]):
                        dTpl2pt = (phTS,opTS,mTag,t0)
                        if existsDataSet_2pt[dTpl2pt]:
                            t0Tag = 't0_%d'%(t0)

                            for ir,irow in enumerate(range(1,nRow+1)):
                                ckeyA = (opT,phT,mTag,isrc,isnk,t0,irow) # All info in one tuple

                                c2ptFileDir  = get2ptFileDir(corr2ptMainDir[(phT,opT)],t0Tag,mFTag)
                                c2ptFileName = get2ptFileName(phT,t0Tag,src,snk,irow,mFTag)               
                                c2ptFile = '%s/%s' % (c2ptFileDir,c2ptFileName)

                                with open(c2ptFile) as fp:
                                    line = fp.readlines()
                                    c = 0
                                    for n in line:
                                        it   = c%Nt
                                        icfg = c//Nt
                                        c2pt_rawAll[icfg,it,ir,it0] = complex(np.float64(n.split()[1]),
                                                                              np.float64(n.split()[2]))
                                        c += 1
                                # Reading file

                                # Jackknife average on All two-point functions
                                c2pt_Allbins[ckeyA], Nbins_2pt_Dict[dTpl] = jackknife_binning(c2pt_rawAll[:,:,ir,it0].real)
                                Nbins_2pt = Nbins_2pt_Dict[dTpl]
                                c2pt_Allave[ckeyA] = jackknife_ave(c2pt_Allbins[ckeyA],Nbins_2pt,tL=Nt)
                            # for-irow

                            # Standard average and standard error (still average over rows)
                            # Might need this for constructing covariance matrix
                            ckeyB = (opT,phT,mTag,isrc,isnk,t0)
                            c2pt_rowAvg[ckeyB] = np.mean(c2pt_rawAll[:,:,:,it0],axis=2,dtype=np.complex128) # Average over rows
                            c2pt_rawAllave[ckeyB] = (np.mean(c2pt_rowAvg[ckeyB],axis=0),
                                                        np.std(c2pt_rowAvg[ckeyB],axis=0)/np.sqrt(Ncfg))
                        else:
                            print('### Skipping dataset:', dTpl2pt)
                    # for-t0

                    # Average over rows and t0
                    c2pt_Tmp1 = np.mean(c2pt_rawAll,axis=3,dtype=np.complex128) # over t0
                    c2pt_Tmp2 = np.mean(c2pt_Tmp1  ,axis=2,dtype=np.complex128) # over rows

                    # Standard average and standard error
                    c2pt_rawAve[ckey] = (np.mean(c2pt_Tmp2,axis=0),np.std(c2pt_Tmp2,axis=0)/np.sqrt(Ncfg))

                    # Jackknife-average on t0- and row-average            
                    c2pt_bins[ckey], Nbins_2pt = jackknife_binning(c2pt_Tmp2.real)
                    c2pt_ave[ckey] = jackknife_ave(c2pt_bins[ckey],Nbins_2pt,tL=Nt)
            # for-src-snk
            print('Done: Momentum %s'%(mTag))
        # for momentum
        
           
print('\nDone\n')

## Construct Ratio of three- and two-point functions

In [ ]:

for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
                
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            srcOpList = srcOpList3pt_Dict[opT][pmTpl] # These are the same in two- and
            snkOpList = srcOpList3pt_Dict[opT][pmTpl] # thre-point functions
        
            for isrc, src in enumerate(srcOpList):
                for isnk, snk in enumerate(snkOpList):
                    ckey2 = (opT,phT,mTag,isrc,isnk) # 2pt-tuple

                    for tsnk in TsinkList[(phT,opT)]:
                        tsnkTag = 'tsnk_%d'%(tsnk)
                        Ntins = tsnk
                        
                        adTpl = (phT,opT,mTag,tsnk)
                        if analyzeDataSet[adTpl]:
                        
                            for gMat in gMatList:
                                insOpName = gMatMap[phT][gMat][0]
                                insOpRow  = gMatMap[phT][gMat][1]

                                for z in dispLen:
                                    ckey3 = (opT,phT,mTag,tsnk,isrc,isnk,gMat,z) # 3pt-tuple


#                                    R[ckey3] = c3pt_bins[ckey3][:,:] / c2pt_bins[ckey2][:,tsnk]
                                

print('\nDone\n')